In [1]:
# add path to explore_by_example src folder
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


# import usual libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import f1_score

# our system imports
from aideme import *

%matplotlib inline

In [2]:
def plot_fscore(metrics):
    df_list = [pd.DataFrame.from_dict({i: metric for i, metric in enumerate(ls)}, orient='index') for ls in metrics]
    avg = sum([df['fscore'][~df['fscore'].isna()] for df in df_list]) / len(df_list)
    avg.plot(ylim=[0,1], marker='o')
    plt.show()

In [3]:
# DUMMY DATA
N = int(1e5)
dim = 2
limit = 2 * (0.001)**(1. / dim)
rng = np.random.RandomState(0)
X = rng.uniform(low=-2, high=2, size=(N, dim))  # do not forget to standardize the data. For this distribution, it should be fine without it.
y_subspace = np.vstack([np.abs(X[:, i]) < limit for i in range(dim)]).T.astype('float')  # partial labels (for each subspace)
y = y_subspace.min(axis=1)
labeled_set = LabeledSet(y, y_subspace)

# visualize data distribution
print('selectivity :', 100 * y.sum() / len(y), '%')

#plt.figure(figsize=(10,8))
#plt.scatter(X[:, 0], X[:, 1], s=0.5, c=['b' if lb else 'r' for lb in y])
#plt.show()

selectivity : 0.105 %


# NO FACTORIZATION EXAMPLE

In [4]:
# SET-UP EXPLORATION CONFIGURATION
NUMBER_OF_ITERATIONS = 100  # number of points to be labeled by the user
SUBSAMPLING = None
INITIAL_SAMPLER = stratified_sampler(labeled_set, pos=1, neg=1)  # start with one random positive sample and one random negative sample
#INITIAL_SAMPLER = random_sampler(10)
CALLBACK = [ # callback functions to be called at the end of each iteration
    classification_metrics(y, 'fscore'), 
    three_set_metric,
]
CALLBACK_SKIP = 10
PRINT_CALLBACK_RESULT = True
CONVERGENCE_CRITERIA = [
    max_iter_reached(NUMBER_OF_ITERATIONS),
    #all_points_are_known,
    #metric_reached_threshold('fscore', 0.8),
    #metric_reached_threshold('tsm', 0.9),
]

explore = PoolBasedExploration(INITIAL_SAMPLER, SUBSAMPLING, CALLBACK, CALLBACK_SKIP, PRINT_CALLBACK_RESULT, CONVERGENCE_CRITERIA)

# CHOOSE AN ALGORITHM
#learner = RandomSampler(SVC(C=1e5, kernel='rbf', gamma='auto'))  # choose a random point
learner = SimpleMargin(C=1024, kernel='rbf')  # choose point closest to SVM decision boundary
#learner = KernelQueryByCommittee(kernel='rbf', sampling='deterministic', n_samples=8, warmup=1000, thin=100, rounding=True)  # version space algorithm
learner = DualSpaceModel(learner, sample_unknown_proba=0.5, mode='positive')  # Dual Space model


# RUN EXPLORATION
metrics = explore.run(X, labeled_set, learner, repeat=1)  # 'repeat' specifies how many times to repeat the exploration process

# COMPUTE AVERAGE F-SCORE OVER ALL REPEATS AND PLOT
#plot_fscore(metrics)
df = pd.DataFrame.from_dict({i: metric for i, metric in enumerate(metrics[0])}, orient='index')
df

iter: 1, fscore: 0.00283076093549909, tsm: 0.0
iter: 11, fscore: 0.028351559335763468, tsm: 0.0
iter: 21, fscore: 0.1563663440059568, tsm: 0.0
iter: 31, fscore: 0.0, tsm: 0.0
iter: 41, fscore: 0.5838509316770186, tsm: 0.00019592476489028212
iter: 51, fscore: 0.6705882352941176, tsm: 0.0018947194991350193
iter: 61, fscore: 0.8571428571428571, tsm: 0.008130081300813009
iter: 71, fscore: 0.8045977011494253, tsm: 0.3
iter: 81, fscore: 0.8823529411764706, tsm: 0.4791666666666667
iter: 91, fscore: 0.970873786407767, tsm: 0.6381578947368421
iter: 101, fscore: 0.9767441860465116, tsm: 0.8292682926829268


,phase,iter_time,labeled_indexes,final_labels,partial_labels,fit_time,get_next_time,fscore,tsm,callback_time
0,begin,0.004387,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,initial_sampling,0.036220,"[75881, 52648]","[1.0, 0.0]","[[1.0, 1.0], [0.0, 0.0]]",0.000986,0.035234,0.002831,0.000000,0.456796
2,exploration,0.036629,[51151],[0.0],"[[0.0, 0.0]]",0.000851,0.035778,NaN,NaN,NaN
3,exploration,0.039443,[76109],[0.0],"[[0.0, 0.0]]",0.000977,0.038466,NaN,NaN,NaN
4,exploration,0.039428,[47975],[0.0],"[[0.0, 0.0]]",0.000733,0.038695,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
97,exploration,0.144438,[99053],[1.0],"[[1.0, 1.0]]",0.009192,0.135246,NaN,NaN,NaN
98,exploration,0.011119,[40021],[1.0],"[[1.0, 1.0]]",0.009785,0.001335,NaN,NaN,NaN
99,exploration,0.003933,[67928],[0.0],"[[0.0, 0.0]]",0.002438,0.001494,NaN,NaN,NaN
100,exploration,0.140606,[58399],[0.0],"[[1.0, 0.0]]",0.002457,0.138149,NaN,NaN,NaN


# FACTORIZATION EXAMPLE

In [5]:
# SET-UP EXPLORATION CONFIGURATION
NUMBER_OF_ITERATIONS = 100  # number of points to be labeled by the user
SUBSAMPLING = None
INITIAL_SAMPLER = stratified_sampler(labeled_set, pos=1, neg=1)  # start with one random positive sample and one random negative sample
#INITIAL_SAMPLER = random_sampler(10)
CALLBACK = [ # callback functions to be called at the end of each iteration
    classification_metrics(y, 'fscore'), 
    three_set_metric,
]
CALLBACK_SKIP = 10
PRINT_CALLBACK_RESULT = True
CONVERGENCE_CRITERIA = [
    max_iter_reached(NUMBER_OF_ITERATIONS),
    #all_points_are_known,
    #metric_reached_threshold('fscore', 0.8),
    metric_reached_threshold('tsm', 1.0),
]

explore = PoolBasedExploration(INITIAL_SAMPLER, SUBSAMPLING, CALLBACK, CALLBACK_SKIP, PRINT_CALLBACK_RESULT, CONVERGENCE_CRITERIA)

# Factorized AL algorithms
PARTITION = [[i] for i in range(dim)]  #[[0], [1]]

# FACTORIZED VERSION SPACE
# label_function = 'AND', 'OR', 'PROD'
# loss = 'GREEDY', 'SQUARED', 'PRODUCT'
#learner = SubspatialVersionSpace(warmup=100, thin=10, n_samples=8, rounding=True, kernel='rbf', gamma=None, partition=PARTITION, label_function='AND', loss='GREEDY')

# FACTORIZED SIMPLE MARGIN
#learner = SubspatialSimpleMargin(C=1024, kernel='rbf', gamma=5, partition=PARTITION, label_function='AND')

# FACTORIZED DSM
learner = DualSpaceModel(SimpleMargin(C=1024, kernel='rbf'), mode=['positive', 'positive'], sample_unknown_proba=0.5, partition=PARTITION)  # Dual Space model


# RUN EXPLORATION
metrics = explore.run(X, labeled_set, learner, repeat=1)  # use y_subspace here

# COMPUTE AVERAGE F-SCORE OVER ALL REPEATS AND PLOT
#plot_fscore(metrics)
df = pd.DataFrame.from_dict({i: metric for i, metric in enumerate(metrics[0])}, orient='index')
df

iter: 1, fscore: 0.0041468375427025535, tsm: 0.0
iter: 11, fscore: 0.24909310761789602, tsm: 0.04172274562584118
iter: 21, fscore: 0.9855072463768116, tsm: 0.7727272727272727
iter: 31, fscore: 0.9767441860465116, tsm: 0.9459459459459459
iter: 41, fscore: 1.0, tsm: 1.0


,phase,iter_time,labeled_indexes,final_labels,partial_labels,fit_time,get_next_time,fscore,tsm,callback_time
0,begin,0.004768,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,initial_sampling,0.384818,"[75796, 79067]","[1.0, 0.0]","[[1.0, 1.0], [0.0, 0.0]]",0.350586,0.034232,0.004147,0.000000,0.445790
2,exploration,0.159165,[24076],[0.0],"[[0.0, 0.0]]",0.142277,0.016887,NaN,NaN,NaN
3,exploration,0.297063,[71312],[0.0],"[[0.0, 0.0]]",0.184701,0.112362,NaN,NaN,NaN
4,exploration,0.230453,[78715],[0.0],"[[0.0, 0.0]]",0.155820,0.074632,NaN,NaN,NaN
5,exploration,0.027193,[13724],[0.0],"[[1.0, 0.0]]",0.024827,0.002367,NaN,NaN,NaN
6,exploration,0.067316,[58766],[0.0],"[[0.0, 0.0]]",0.028773,0.038543,NaN,NaN,NaN
7,exploration,0.003008,[30486],[0.0],"[[1.0, 0.0]]",0.002142,0.000866,NaN,NaN,NaN
8,exploration,0.040416,[61572],[1.0],"[[1.0, 1.0]]",0.001162,0.039254,NaN,NaN,NaN
9,exploration,0.001965,[12464],[0.0],"[[1.0, 0.0]]",0.001116,0.000849,NaN,NaN,NaN
